<a href="https://colab.research.google.com/github/elainedias16/TCC/blob/main/Roteiro_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ******* LIBRARIES **********

# Basic libraries for data manipulation
import pandas as pd
from bs4 import BeautifulSoup

# Libraries used for reading database
from posixpath import defpath
import requests
import os
import chardet

# Libraries used for preprocessing
import nltk
import re
import unicodedata
nltk.download('stopwords')
from nltk.corpus import stopwords
## Stemming
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('rslp')
## Lemmanization
import spacy
!python -m spacy download pt_core_news_sm

In [ ]:
def remove_tags(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)


# Inicializar o stemmer RSLPStemmer
# RSLP is a pt stemmer, Porter is one of the most used en stemmers
stemmer = RSLPStemmer()
def stemming(text):
  words = word_tokenize(text)
  stemmed_words = [stemmer.stem(word) for word in words]
  stemmed_sentence = ' '.join(stemmed_words)
  return stemmed_sentence


# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('pt_core_news_sm')
def lemmanization(text):
  doc = nlp(text)
  lemmatized_tokens = [token.lemma_ for token in doc]
  lemmatized_text = ' '.join(lemmatized_tokens)
  return lemmatized_text


In [ ]:
stop_words = set(stopwords.words('portuguese'))

def pre_processing(text):
  #Put space in non alphanumeric characteres
  text_without_non_alnum = ''
  for character in text:
    if character.isalnum():
      text_without_non_alnum += character
    else:
      text_without_non_alnum += ' '

  # Remove accents:
  text_without_accents = ''.join(c for c in unicodedata.normalize('NFD', text_without_non_alnum) if unicodedata.category(c) != 'Mn')

  # Remove duplicated spaces:
  text_without_duplicates = ' '.join(text_without_accents.split())

  # Lowercasting
  text_without_duplicates = text_without_duplicates.lower()

  # Remove stop words:
  words = nltk.word_tokenize(text_without_duplicates)
  text_without_stop_words = ''
  for word in words:
    if word not in stop_words:
      text_without_stop_words += word + ' ' #It's necessary to put space because tokenize takes away it

  # Lemmanization
  lemmatized_text = lemmanization(text_without_stop_words)

  # Stemming:
  stemmed_text = stemming(lemmatized_text)

  return stemmed_text


In [ ]:
id_path = '1TJTEKu5hxk25q36O6xF1OruLjySlW6_n'
directory_path = '/content/drive/MyDrive/dados-aTribuna-classes-selecionadas'

def create_dataframe():
  df = pd.DataFrame(columns=['genre', 'content'])
  folders = os.listdir(directory_path)

  for folder in folders:
    genre_path = directory_path + '/' + folder

    if os.path.isdir(genre_path): #There is a file in the root
        os.chdir(genre_path)
        for filename in os.listdir(genre_path):
            file_path = os.path.join(genre_path, filename)

            if filename.endswith('.txt'):
              with open(file_path, 'rb') as file:
                content = file.read()
                content_without_html_tags = remove_tags(content)
                content_pre_procesed = pre_processing(content_without_html_tags)
                # df = df.append({'genre': folder, 'content': content_pre_procesed}, ignore_index=True) #Append method will be deprecied soon
                df_to_concat = pd.DataFrame({'genre': folder, 'content': content_pre_procesed}, index=[0])
                df = pd.concat([df, df_to_concat], ignore_index=True)


  return df


df = create_dataframe()

df.head()

,genre,content
0,inf,raceh cad vez perig need sped carbon promet in...
1,inf,certificaco profiss competi merc ti al experie...
2,inf,web famili intern brasil pass naveg sit relac ...
3,inf,brasil us web jog on lin pagin hum pi sao tem ...
4,inf,pequen mund panasonic lanc pequen cam digit mu...
